# McRAPTOR for latest departure problem 

*Authors: Felix Grimberg, Cyril Pulver*

Multiple-criteria adaptation of RAPTOR (Round bAsed Public Transit Optimized Router) algorithm (cf. [README](../README.md)).

## Left out at this stage:

- Realistic time to get out of one transport and walk to the platform of the next. Instead, we just set it to 2 minutes, no matter what.
We also reserve 2 minutes for walking from the entrance of a station to the correct platform, and vice versa, when walking between stations (total: `time_to_walk_distance + 4 minutes`)

# Run the algorithm

Please make sure that you follow these steps:
1. Execute `git lfs pull` in a terminal to download our data files (if you haven't already).
2. **Execute the code cells in the other sections of this notebook ("Load the data") and onwards to load the data and define the classes and functions that make up the algorithm.**
3. Execute all three cells below (grid, log and output)
4. Plan journeys !

After executing the two cells below, please enter:
- The departure stop (only BPUIC-like stop ID are currently supported, with a minimum of 7 characters. Platform information etc. is tolerated, but ignored)
- The arrival (target) stop (same format)
- The arrival time (format HH:mm)
- The minimum probability of success for the journey (slider)

In [ ]:
# Execute to set input
grid

In [ ]:
log

In [ ]:
# execute to get output
output

## Load the data
### General considerations
We adhere to the data structures proposed by Delling et al. These structures aim to minimize read times in memory by making use of consecutive in-memory adresses. Thus, structures with varying dimensions (e.g dataframes, python lists) are excluded. We illustrate the difficulty with an example. 

Each route has a potentially unique number of stops. Therefore, we cannot store stops in a 2D array of routes by stops, as the number of stops is not the same for each route. We adress this problem by storing stops consecutively by route, and keeping track of the index of the first stop for each route.

This general strategy is applied to all the required data structures, where possible.

### routes
The `routes` array will contain arrays `[n_trips, n_stops, pt_1st_stop, pt_1st_trip]` where all four values are `int`. To avoid overcomplicating things and try to mimic pointers in python, `pt_1st_stop` and `pt_1st_trip` contain integer indices.

In [ ]:
import numpy as np
import pickle

def pkload(path):
    with open(path, 'rb') as f:
        obj = pickle.load(f)
    return obj

In [ ]:
routes = pkload("../data/routes_array_cyril.pkl").astype(np.uint32)
print(routes.shape)
routes

### routeStops
`routeStops` is an array that contains the ordered lists of stops for each route. `pt_1st_stop` in `routes` is required to get to the first stop of the route. is itself an array that contains the sequence of stops for route $r_i$.

In [ ]:
routeStops = pkload("../data/route_stops_array_cyril.pkl").astype(np.uint16)
print(routeStops.shape)
print(routeStops.max())
routeStops

### stopTimes

The i-th entry in the `stopTimes` array is itself an array which contains the arrival and departure time at a particular stop for a particular trip. `stopTimes` is sorted by routes, and then by trips. We retrieve the index of the first (earliest) trip of the route with the pointer `pt_1st_trip` stored in `routes`. We may use the built-in `numpy` [date and time data structures](https://blog.finxter.com/how-to-work-with-dates-and-times-in-python/). In short, declaring dates and times is done like this: `np.datetime64('YYYY-MM-DDThh:mm')`. Entries with a `NaT` arrival or departure times correspond to beginning and end of trips respectively.

Note that trips are indexed implicitely in stopTimes, but we decided to change a little bit from the paper and index them according to their parent route instead of giving them an absolute index. It makes things a bit easier when coding the algorithm.

In [ ]:
stopTimes = pkload("../data/stop_times_array_cyril.pkl")
print(stopTimes.shape)
stopTimes

`NaT` is the `None` equivalent for `numpy datetime64`.

In [ ]:
print(np.isnat(stopTimes[0]))

### stopRoutes

`stopRoutes` contains the routes (as `int`s representing an index in `routes`) associated with each stop. We need the pointer in `stops` to index `stopRoutes` correctly.

In [ ]:
stopRoutes = pkload("../data/stop_routes_array_cyril.pkl").flatten().astype(np.uint32)
print(stopRoutes.shape)
stopRoutes

### transfers
`transfers` is a 2D `np.ndarray` where each entry `[p_j, time]` represents (in seconds) the time it takes to walk from stop p_j to the implicitely given stop p_i.
p_i is given implicitely by the indexing, in conjunction with `stops`. In other words:
`transfers[stops[p_i][2]:stops[p_i][3]]` returns all the footpaths arriving at stop p_i.

As we cannot store different data types in numpy arras, `time` will have to be converted to `np.timedelta64`, the format used to make differences between `np.datetime.64` variables. We will consider all `time` values as **positive values in seconds**.

In [ ]:
transfers = pkload("../data/transfer_array_cyril.pkl").astype(np.uint16)
print(transfers.shape)
transfers

### stops

`stops` stores the indices in `stopRoutes` and in `transfers` corresponding to each stop.

`stopRoutes[stops[p][0]:stops[p][1]]` returns the routes serving stop p.

`transfers[stops[p][2]:stops[p][3]]` returns the footpaths arriving at stop p.

In [ ]:
stops = pkload("../data/stops_array_cyril.pkl")
stopRoutes = pkload("../data/stop_routes_array_cyril.pkl")
print(np.isnan(stops.astype(np.float64)).sum(axis=0))
print(np.equal(stops, None).sum(axis=0))
print(stops.shape)
stops = stops[:,[0,0,1,1]]
# Make column 1 contain the start_index of the next stop in stopRoutes
stops[:-1,1] = stops[1:,0]
stops[-1, 1] = stopRoutes.shape[0]
# Deal with NaN's in column 2 (for stops with 0 foot transfers within 500m)
if np.isnan(stops[-1,2]).item():
    stops[-1,2] = transfers.shape[0]
for i in np.isnan(stops[:-1,2].astype(np.float64)).nonzero()[0][::-1]:
    stops[i,2] = stops[i+1,2]
print(np.isnan(stops.astype(np.float64)).sum(axis=0))
# Make column 3 contain the start_index of the next stop in stopRoutes
stops[:-1,3] = stops[1:,2]
stops[-1, 3] = transfers.shape[0]
# Convert to int
stops = stops.astype(np.uint32)
stops

### Example

In [ ]:
p = 0
routes_serving_p = stopRoutes[stops[p][0]:stops[p][1]]
print("routes serving stop 0:", routes_serving_p)
for r in routes_serving_p:
    print("stops of route {}:".format(r), routeStops[routes[r][2]:routes[r][2]+routes[r][1]])
for pPrime, walking_seconds in transfers[stops[p][2]:stops[p][3]]:
    print("stop {} can be reached from stop {} by walking for {} seconds.".format(p, pPrime, walking_seconds))

### Distribution of delays

In [ ]:
import gzip 
with gzip.open("../data/transfer_probabilities.pkl.gz") as distrib_pkl:
    distrib_delays = pickle.load(distrib_pkl)
    
distrib_delays[0:2]

### Relate `stop_id`s and `trip_headsign`s to the integer indices used in the algorithm

In [ ]:
stop_times_df = pkload("../data/stop_times_df_cyril.pkl")[['route_id', 'stop_id_general', 'stop_name', 'trip_headsign', 'route_int', 'stop_int', 'route_desc']]
stop_times_df.head()

In [ ]:
stop_ids_names = stop_times_df[['stop_id_general', 'stop_int', 'stop_name']].drop_duplicates()
assert np.all(stop_ids_names == stop_ids_names.drop_duplicates(subset='stop_int'))
assert np.all(stop_ids_names == stop_ids_names.drop_duplicates(subset='stop_id_general'))
assert np.all(stop_ids_names == stop_ids_names.drop_duplicates(subset='stop_name'))
stop_ids_names = stop_ids_names.sort_values(by='stop_int')
stop_ids = stop_ids_names['stop_id_general'].to_numpy()
stop_names = stop_ids_names['stop_name'].to_numpy()
print(stop_ids, stop_names)

In [ ]:
p = np.random.randint(stops.shape[0])
print(p, stop_ids[p], stop_names[p])
stop_times_df[stop_times_df['stop_int'] == p].head(2)

# Define the classes and functions making up the journey planner

Based on a modified version of RAPTOR (reversed RAPTOR), we implement a multiple criteria RAPTOR algorithm.
The optimization criteria are:
- Latest departure
- Highest probability of success of the entire trip
- Lowest number of connections (implicit with the round-based approach)

We specify the time to change platforms as an absolute constant:

In [ ]:
tau_change_platform = np.timedelta64(2, 'm')
np.datetime64('2020-05-11T15:30') - tau_change_platform

Next, we define some shorthand functions to help with the indexing in the data structures:

In [ ]:
def gen_random_stop_id():
    """Generate a random stop_id to test the journey planner."""
    return str(stop_ids[np.random.randint(stops.shape[0])])+':0:5DE'

def stop_id_to_int(p_id):
    """Given a stop id, returns the corresponding stop_int"""
    return np.asarray(stop_ids == int(p_id[:7])).nonzero()[0].item()

def get_stops(r):
    """Returns the stops of route r"""
    idx_first_stop = routes[r][2]
    return routeStops[idx_first_stop:idx_first_stop+routes[r][1]] # n_stops = routes[r][1]

def calc_stopTimes_idx(r, t, offset_p):
    """Returns the index of the entry in stopTimes
    corresponding to the offset_p-th stop of the t-th trip
    of route r.
    """
    return (routes[r][3]       # 1st trip of route
            + t * routes[r][1] # offset for the right trip
            + offset_p         # offset for the right stop
           )

def get_arrival_time(r, t, offset_p):
    """Returns 2000 (instead of 0) if t is None.
    Otherwise, returns the arrival time of the t-th trip of route r
    at the offset_p-th stop of route r.
    trips and stops of route r start at t=0, offset_p=0.
    """
    if t is None:
        return np.datetime64('2000-01-01T01:00')
    
    return stopTimes[calc_stopTimes_idx(r,t,offset_p)][0] # 0 for arrival time

def get_departure_time(r, t, offset_p):
    """Throws TypeError if t is None.
    Otherwise, returns the departure time of the t-th trip of route r
    at the offset_p-th stop of route r.
    trips and stops of route r start at t=0 & offset_p=0.
    """
    if t is None:
        raise TypeError("Requested departure time of None trip!")
    
    return stopTimes[calc_stopTimes_idx(r,t,offset_p)][1] # 1 for departure time

# This one isn't indexing related. Forgive us.
def time2str(t):
    """Prints the hour and minutes of np.datetime64 t."""
    return str(t.astype('datetime64[m]')).split('T')[1]

In the cell below, we define the following class hierarchy of label types:
```
BaseLabel
 +-- RouteLabel
 +-- ImmutableLabel
      +-- WalkLabel
      +-- TargetLabel
```
Labels come together recursively to represent a (partial) solution, going from the `stop` attribute of the first label to the target stop (that stop will be referred to as "here").
Each partial solution is associated with a departure time `tau_dep`, an overall probability of success (all the way to the target stop) `Pr`, and a number of trips necessary to get from here to the target stop.
Any two labels can be compared via their `dominates` method, to retain only non-dominated labels.
Further, all types of labels implement a `print_instructions` method, used to recursively print instructions for the entire journey.
Each `RouteLabel` corresponds to the portion of a journey that begins with standing on the correct platform to board a vehicle, and ends with exiting from that vehicle at another stop.
Each `WalkLabel` corresponds to the portion of a journey that begins with walking away from the main hall of one stop and ends with having walked all the way to the main hall of another stop.

In [ ]:
class InstantiationException(Exception):
    pass

class BaseLabel:
    """An abstract base class for Labels. Do not instantiate.
    A label corresponds to a recursive (partial) solution, going
    to the target stop from the stop currently under consideration.
    """
    indent = " "*4
    def __init__(self, stop, tau_dep, Pr, n_trips):
        self.stop    = stop
        self.tau_dep = tau_dep
        self.Pr      = Pr
        self.n_trips = n_trips
    
    def dominates(self, other):
        """Returns True if self dominates other, else returns False.
        other: another Label instance.
        """
        if  self.tau_dep >= other.tau_dep \
        and self.Pr      >= other.Pr      \
        and self.n_trips <= other.n_trips :
            return True
        return False
    
    def stop2str(self):
        """Returns a printable str representing self.stop"""
        return "{stopN} (stop {stopI})".format(
                    stopN = stop_names[self.stop],
                    stopI = stop_ids[self.stop]
        )
        
    def print_journey(self):
        print("Departure stop:        ", self.stop2str())
        print("Departure time:        ", time2str(self.tau_dep))
        print("Number of trips used:  ", self.n_trips)
        print("Probability of success: {:.4f}".format(self.Pr))
        
        self.print_instructions()
    
    def to_str(self):
        s = "Departure at {0} from stop {1} (id: {2}, int: {3}).".format(
            self.tau_dep,
            stop_names[self.stop],
            stop_ids[self.stop],
            self.stop
        )
        return repr(type(self)) + s
    
    def pprint(self, indent=''):
        print(indent, self.to_str())
    
    def copy(self):
        raise InstantiationException("class BaseLabel should never "
                                     "be instantiated."
                                    )

class ImmutableLabel(BaseLabel):
    """Base class for immutable Labels"""
    def copy(self):
        return self

class TargetLabel(ImmutableLabel):
    """A special type of label reserved for the target stop."""
    def __init__(self, stop, tau_dep):
        BaseLabel.__init__(self, stop, tau_dep, 1., 0)
        
    def print_instructions(self):
        """Finish printing instructions for the journey."""
        print("Target stop:           ", self.stop2str())
        print("Requested arrival time:", time2str(self.tau_dep))

class WalkLabel(ImmutableLabel):
    """A special type of label for walking connections."""
    def __init__(self, stop, tau_walk, next_label):
        """Create a new WalkLabel instance.
        stop:       stop where you start walking.
        tau_walk:   (np.timedelta64) duration of the walk.
        next_label: label describing the rest of the trip after walking.
        """
        if isinstance(next_label, WalkLabel):
            raise ValueError("Cannot chain two consecutive WalkLabels!")
        tau_dep = next_label.tau_dep - tau_walk - tau_change_platform
        BaseLabel.__init__(self, stop, tau_dep, next_label.Pr, next_label.n_trips)
        self.tau_walk   = tau_walk
        self.next_label = next_label
        
    def print_instructions(self):
        """Recursively print instructions for the whole journey."""
        print(self.indent + "Walk {:.1f} minutes from".format(
                self.tau_walk / np.timedelta64(1,'m')
                ),
              self.stop2str()
             )
        print(self.indent*2 + "to", self.next_label.stop2str())
        self.next_label.print_instructions()

class RouteLabel(BaseLabel):
    """A type of label for regular transports."""
    def __init__(self,
                 tau_dep,
                 r,
                 t,
                 offset_p,
                 next_label,
                 Pr_connection_success
                ):
        
        self.tau_dep      = tau_dep
        self.r            = r
        self.t            = t
        self.offset_p_in  = offset_p
        self.offset_p_out = offset_p
        self.next_label   = next_label
        # Store Pr_connection_success for self.copy()
        self.Pr_connection_success = Pr_connection_success
        
        self.route_stops = get_stops(self.r)
        self.stop        = self.route_stops[self.offset_p_in]
        self.Pr          = self.Pr_connection_success * self.next_label.Pr
        self.n_trips     = self.next_label.n_trips + 1
    
    def update_stop(self, stop):
        self.stop     = stop
        self.offset_p_in = self.offset_p_in - 1
        # Sanity check:
        assert self.offset_p_in >= 0
        assert self.route_stops[self.offset_p_in] == stop
        self.tau_dep  = get_departure_time(self.r, self.t, self.offset_p_in)
        
    def print_instructions(self):
        """Recursively print instructions for the whole journey."""
        stopTimes_idx = calc_stopTimes_idx(self.r, self.t,
                                           self.offset_p_in)
        
        print(self.indent   + "At", self.stop2str())
        print(self.indent*2 + "take the",
              stop_times_df['route_desc'][stopTimes_idx], "to",
              stop_times_df['trip_headsign'][stopTimes_idx]
             )
        print(self.indent*2 + "leaving at", time2str(self.tau_dep),
              "(route id: {}).".format(
                  stop_times_df['route_id'][stopTimes_idx]
                  )
             )
        
        tau_arr = get_arrival_time(
                    self.r,
                    self.t,
                    self.offset_p_out
                    )
        assert self.next_label.stop == self.route_stops[self.offset_p_out]
        
        print(self.indent   + "Get out at", self.next_label.stop2str())
        print(self.indent*2 + "at", time2str(tau_arr)+".")

        self.next_label.print_instructions()
        
    def copy(self):
        """When RouteLabels are merged into the bag of a stop,
        they must be copied (because they will subsequently
        be changed with self.update_stop()).
        """
        l = RouteLabel(self.tau_dep,
                       self.r,
                       self.t,
                       self.offset_p_in,
                       self.next_label,
                       self.Pr_connection_success
                      )
        l.offset_p_out = self.offset_p_out
        return l

The functions below define the bag merge operation and implement the journey planner:

In [ ]:
def run_mc_raptor(stop_id_start,
                  stop_id_destination,
                  arrival_time_max,
                  Pr_min
                 ):
    """Run MC RAPTOR, using the data defined in cells above (stopRoutes etc.).
    Inputs:
        stop_id_start:       source stop id
                             str (format: '1234567')
        stop_id_destination: target stop id
                             str (format: '1234567')
        arrival_time_max:    latest acceptable arrival time
                             str (format: '14:56')
        Pr_min:              minimum acceptable probability of success
                             float in open interval (0,1)
    Outputs:
        bags_p_s: bags_p_s[k] contains the Pareto set of non-dominated journeys
                  from stop_id_start to stop_id_destination that use at most
                  k different trips (i.e, getting in at most k different vehicles),
                  under the given constraints:
                      1. Each journey must succeed with a probability
                         greater or equal to Pr_min.
                      2. The journey is a success if and only if all individual
                         connections succeed, including the user's appointment
                         in p_t at tau_0.
                      3. A connection succeeds if, and only if, the user reaches
                         the platform before or on the scheduled departure time
                         (allowing some time to change platforms)
                  Non-dominated:
                      A journey J1 is *dominated* by another journey J2, if
                      J2 departs no earlier than J1 AND the probability of
                      success of J2 is no less than that of J1.
                  Pareto set:
                      Each bag in bags_p_s contains only journeys that are not
                      dominated by any other possible journey. Such a collection
                      of non-dominated solutions is called a *Pareto set*.
        bags:     bags[k][stop_id_to_int(stop_id)] contains the Pareto set of
                  non-dominated journeys from stop_id to stop_id_destination
                  that use at most k different trips under the given constraints.
                  
    Each journey is represented as a Label that forms the start of a chain.
    The journey can be reconstructed by calling label.print_journey().
    """
# Input sanitization:
    try:
        tau_0 = np.datetime64('2020-05-24T'+arrival_time_max)
        p_s   = stop_id_to_int(stop_id_start)
        p_t   = stop_id_to_int(stop_id_destination)
    except Exception as e:
        print("ERROR parsing input! Please make sure that:")
        print("- stop_id_start and stop_id_destination are strings "
              "beginning with 7 or more digits.")
        print("- arrival_time_max is in the format 'hh:mm'. Example: '09:42'.")
        raise e
    
# initialization
    k_max = 10 # Maximum number of rounds
    
    # For each route and for each label at each stop p, we will look at the n latest
    # trips until we find a trip for which the individual connection at stop p
    # succeeds with a probability at least equal to Pr_threshold.
    # Under some reasonable assumptions, setting Pr_threshold = Pr_min**(1/k)
    # guarantees that we will find a solution, if a solution exists involving at
    # most k connections (including the user's appointment in p_t at tau_0).
    Pr_threshold = Pr_min**(0.1)
    
    # Initialize empty bags for each stop for round 0:
    n_stops = stops.shape[0]
    bags = [
        [
            [] # an empty bag
        for _ in range(n_stops)] # one empty bag per stop
    ]

    # Create a TargetLabel for p_t, and mark p_t
    bags[0][p_t].append(TargetLabel(p_t, tau_0))
    marked = {p_t}

    print("Searching for journeys from",
          stop_names[p_s],
          "(stop {})".format(stop_ids[p_s]),
          "to", bags[0][p_t][0].stop2str())

# Define bag operations (they depend on p_s and Pr_min for target pruning):
    def update_bag(bag, label, k, target_pruning=True):
        """Add label to bag and remove dominated labels.
        bag is altered in-place.

        k: Round number, used for target pruning.

        returns: Boolean indicating whether bag was altered.
        """
        if target_pruning:
            # Apply the Pr_min constraint to label:
            if label.Pr < Pr_min:
                return False

            # Prune label if it is dominated by bags[k][p_s]:
            for L_star in bags[k][p_s]:
                if L_star.dominates(label):
                    return False

        # Otherwise, merge label into bag1
        changed = False
        for L_old in bag:
            if L_old.dominates(label):
                return changed
            if label.dominates(L_old):
                bag.remove(L_old)
                changed = True
        bag.append(label.copy())
        return True

    def merge_bags(bag1, bag2, k, target_pruning=True):
        """Merge bag2 into bag1 in-place.
        k: Round number, used for target pruning.
        returns: Boolean indicating whether bag was altered.
        """
        changed = False
        for label in bag2:
            changed = update_bag(bag1, label, k, target_pruning) or changed
        return changed
    
    globals().update({'merge_bags': merge_bags, 'Pr_min':Pr_min})
    
# Define the footpaths-checking function (depends on update_bag)
    def check_footpaths(bags, marked, k):
        """Modify bags and marked in-place to account for foot-paths."""
        q = []
        for p in marked:
            for pPrime, delta_seconds in transfers[stops[p][2]:stops[p][3]]:
                q.append((p, pPrime, delta_seconds))
        for p, pPrime, delta_seconds in q:
            for L_k in bags[k][p]:
                # We do not allow two consecutive walking trips
                if not isinstance(L_k, WalkLabel):
                    L_new = WalkLabel(pPrime, np.timedelta64(delta_seconds, 's'), L_k)
                    if update_bag(bags[k][pPrime], L_new, k):
                        marked.add(pPrime)

# main loop
    indent= ' '*4

    k = 0
    # Check footpaths leading to p_t at k=0:
    check_footpaths(bags, marked, k)
    while k < k_max:
        k += 1 # k=1 at fist round, as it should.

        # Instead of using best bags, carry over the bags from last round.
    #     if len(bags <= k):

        bags.append([bags[-1][p].copy() for p in range(n_stops)])

        print('\n', ' '*30, 'STARTING round k =', k)
        if len(marked) < 50:
            print('Marked stops at the start of the round:', [stop_ids[p] for p in marked])
        else:
            print('Number of marked stops at the start of the round:', len(marked))
        
        # accumulate routes serving marked stops from previous rounds
        q = []
        for p in marked:
            for r in stopRoutes[stops[p][0]:stops[p][1]]: # foreach route r serving p
                append_r_p = True
                for idx, (rPrime, pPrime) in enumerate(q): # is there already a stop from the same route in q ?
                    if rPrime == r:
                        append_r_p = False
                        p_pos_in_r = np.where(get_stops(r) == p)[0][-1]
                        pPrime_pos_in_r = np.where(get_stops(r) == pPrime)[0][-1]
                        if p_pos_in_r > pPrime_pos_in_r:
                            q[idx] = (r, p) # substituting (rPrime, pPrime) by (r, p)
                if append_r_p:
                    q.append((r, p))
        marked.clear() # unmarking all stops
#         print("Queue:", q)

#         print('Queue before traversing each route: {}'.format(q))
        # traverse each route
        for (r, p) in q:
#             print('\n****TRAVERSING ROUTE r={0} from stop p={1}****'.format(r, p))
            B_route = [] # new empty route bag

            # we traverse the route backwards (starting at p, not from the end of the route)
            stops_of_current_route = get_stops(r)
#             print('Stops of current route:', stops_of_current_route)
            offset_p = np.asarray(stops_of_current_route == p).nonzero()[0]
            if offset_p.size < 1:
                print("WARNING: route {r} is said to serve stop {p} in stopRoutes, but stop {p} "
                      "is not included as a stop of route {r} in routeStops...".format(p=p, r=r))
                offset_p = -1
            else:
                offset_p = offset_p[-1]
            for offset_p_i in range(offset_p, -1, -1):
                p_i = stops_of_current_route[offset_p_i]
#                 print('\n\n'+indent+"p_i: {}".format(p_i))

                # Update the labels of the route bag:
                for L in B_route:
                    L.update_stop(p_i)

                # Merge B_route into bags[k][p_i]
                if merge_bags(bags[k][p_i], B_route, k):
#                     print("marking stop", p_i)
                    marked.add(p_i)

                # Can we step out of a later trip at p_i ?
                # This is only possible if we already know a way to get from p_i to p_t in < k vehicles
                # (i.e., if there is at least one label in bags[k-1][p_i])
                for L_k in bags[k-1][p_i]:
                    # Note that k starts at 1 and bags[0][p_t] contains a TargetLabel.
#                     print('\n'+indent+'----scanning arrival times for route r={0} at stop p_i={1}----'.format(r, p_i))

                    # We check the trips from latest to earliest
                    for t in range(routes[r][0]-1, -1, -1): # n_trips = routes[r][0]
                        # Does t_r arrive early enough for us to make the rest 
                        # of the journey from here (tau[k-1][p_i])?
                        tau_arr = get_arrival_time(r, t, offset_p_i)
#                         print(indent+'arrival time: ', tau_arr)
                        if tau_arr <= L_k.tau_dep - tau_change_platform:

                            max_delay = L_k.tau_dep - tau_arr - tau_change_platform
                            max_delay_int = min(max_delay.astype('timedelta64[m]').astype('int'), 30)
                            
                            Pr_connection = distrib_delays[calc_stopTimes_idx(r, t, offset_p_i),
                                                           max_delay_int + 1]
#                             print(Pr_connection)
                            L_new = RouteLabel(get_departure_time(r, t, offset_p_i),
                                               r,
                                               t,
                                               offset_p_i,
                                               L_k,
                                               Pr_connection
                                              )
                            update_bag(B_route, L_new, k)#:
#                                 print(indent+"Explored connection from")
#                                 L_new.pprint(indent*2)
#                                 print(indent+"to")
#                                 L_k.pprint(indent*2)

                            # We don't want to add a label for every trip that's earlier than tau_dep.
                            # Instead, we stop once we've found a trip that's safe enough.
                            if Pr_connection > Pr_threshold:
                                break
        
#         print(marked)
        # Look at foot-paths (bags and marked are altered in-place):
        check_footpaths(bags, marked, k)
#         print(marked)
        
        # Report the number of new journeys found this round:
        n_new_journeys = len(bags[k][p_s]) - len(bags[k-1][p_s])
        if n_new_journeys:
            print("Found", n_new_journeys, "non-dominated journeys with", k, "trips.")
            # If there's a journey with X trips, we won't look for a solution with X+3 trips:
            if not bags[k-1][p_s]:
                # The condition above means we found a journey for the first time.
                print("Will search for journeys with up to", k+1, "trips.")
                k_max = k+1

        # Additional stopping criterion: reached equilibrium
        if not marked:
            print("\n", ' '*15, "*"*15, " THE END ", "*"*15)
            print("Equilibrium reached with" + "out"*(not bags[k][p_s]),
                  "finding a solution. The end.")
            return [bags[K][p_s] for K in range(len(bags))], bags
    
    # We have exited the while-loop because k==kmax:
    print("\n" + "*"*15 + " THE END " + "*"*15)
    if bags[k][p_s]:
        print("There is a solution with", k-1, "trips.")
    else:
        print("There are no solutions with up to", k, "trips.")
    print("We shall not search for journeys with", k+1, "or more trips.")

    return [bags[K][p_s] for K in range(len(bags))], bags

def time_sorted_pareto(bags_p):
    """Input: list of bags, e.g. for one stop and various k.
              It is assumed that Pr >= Pr_min for each label.
    Output: A Pareto set of non-dominated labels (np.array),
            sorted by decreasing departure time.
    """
    res_bag = []
    for bag in bags_p:
        globals()['merge_bags'](res_bag, bag, 0, target_pruning=False)
    res = np.array(res_bag)
    return res[np.argsort([label.tau_dep for label in res])[::-1]]

def print_solutions(bags_p):
    print("Showing Pareto set of non-dominated journeys")
    if not bags_p[-1]:
        print("There are no journeys to print.")
        return
    L_s = bags_p[-1][0]
    L_t = L_s
    while not isinstance(L_t, TargetLabel):
        L_t = L_t.next_label
    print("from", L_s.stop2str(), "to", L_t.stop2str())
    print("with criteria:")
    print("  - maximize departure time")
    print("  - maximize probability of success")
    print("  - minimize number of individual trips")
    print("and constraints:")
    print("  - arrival time at least", tau_change_platform,
              "minutes before", time2str(L_t.tau_dep), "(2 minutes to walk from the platform to the extraction point).")
    print("  - probability of success at least {Pr_min:.4f}.".format(**globals()))
    print("\nJourneys are sorted by descending departure time.")
    
    for i, label in enumerate(time_sorted_pareto(bags_p)):
        print('\n'*2, '-'*10, 'OPTION', i+1, '-'*10)
        label.print_journey()

In [ ]:
# basic user interface
style_wi_desc = {'description_width': 'initial'}
import ipywidgets as widgets
from ipywidgets import HBox, Label
departure_stop_input = widgets.Text(description='departure stop ID',
                                    style = style_wi_desc
                                   )
arrival_stop_input = widgets.Text(description='arrival stop ID',
                                  style = style_wi_desc
                                 )
arrival_time_input = widgets.Text(description = 'arrival time (HH:mm)',
                                  value='17:30',
                                  style = style_wi_desc
                                 )

probability_min_input = widgets.FloatSlider(
    value=0.9,
    min=0,
    max=1.0,
    step=0.01,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style = style_wi_desc
)

from IPython.display import display
find_journey_button = widgets.Button(description="Find journey")
random_stops_button = widgets.Button(description="Pick random stops")
log    = widgets.Output()
output = widgets.Output()

def on_find_journey_button_clicked(b):
    log.clear_output()
    output.clear_output()
    with log:
        bags_p_s, bags = run_mc_raptor(departure_stop_input.value,
                               arrival_stop_input.value,
                               arrival_time_input.value,
                               probability_min_input.value,
                                      )
    with output:
        print_solutions(bags_p_s)
        
find_journey_button.on_click(on_find_journey_button_clicked)

def on_random_stops_clicked(b):
    departure_stop_input.value = gen_random_stop_id()[:7]
    arrival_stop_input.value   = gen_random_stop_id()

random_stops_button.on_click(on_random_stops_clicked)

from ipywidgets import GridspecLayout
grid = GridspecLayout(4, 2)
grid[0, 0] = departure_stop_input
grid[0, 1] = arrival_stop_input
grid[1, 0] = arrival_time_input
grid[2, :] = HBox([Label('Minimum probability of success'), probability_min_input])
grid[3, 0] = find_journey_button
grid[3, 1] = random_stops_button